## Part 1: Checking GPU Availability

Kahusus untuk kode berikut:
- **nvidia-smi**: Menampilkan informasi GPU (NVIDIA Graphics Processing Unit) yang tersedia di sistem. Perintah ini berguna untuk memverifikasi bahwa GPU dapat digunakan untuk akselerasi komputasi.

In [ ]:
# Menampilkan informasi GPU yang tersedia
!nvidia-smi

: 

# Environment Preparation

In [ ]:
from IPython.display import clear_output
from IPython.display import display, Image
from IPython.display import Image
# IPython.display untuk menampilkan output gambar
import os
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
import numpy as np


## Part 3: Mengimpor Library dan Melakukan Setup

Bagian ini mengimpor semua library yang diperlukan untuk proyek:
- **IPython.display**: Untuk menampilkan output gambar dan membersihkan output
- **cv2 (OpenCV)**: Library untuk processing video dan gambar
- **PIL (Python Imaging Library)**: Untuk manipulasi gambar
- **numpy**: Array dan operasi numerik
- **warnings**: Untuk mengontrol pesan warning

In [ ]:
# Part 2: Menetapkan direktori kerja
# HOME = '/workspace'
HOME = '/kaggle/working'

%cd {HOME}

In [ ]:
!pip install ultralytics
clear_output()

from ultralytics import YOLO
!yolo checks

In [ ]:
# YOlO modelling only support numpy version 1.x
!pip uninstall numpy -y
!pip install "numpy<2.0"

In [ ]:
!pip install gdown

In [ ]:
# Download best.pt (YOLO12s yang udah di training pakai dataset sawit asli)
!gdown 'https://drive.google.com/uc?export=download&id=1hNEVSduMLdJqCn5W4DQXuhMamsS3nB-Z'

## Part 4: Model Loading dan Inferensi

Bagian ini mendownload dan menggunakan model YOLO yang sudah di-training:
- **best.pt**: Model YOLO12s yang telah dilatih dengan dataset sawit asli

# Modelling

In [ ]:
# Download video 1 (untuk mengetes hasil object counting)
!gdown "https://drive.google.com/uc?export=download&id=1w8BACPtro7X3LnLthE6JT65mSiZcL3qc"

## Part 5: Object Counting

Bagian ini melakukan counting FFB (Fresh Fruit Bunch) dari video drone:
- **Phase 5.1**: Download video test dan setup tracker parameters
- **Phase 5.2**: Program pertama - counting total FFB
- **Phase 5.3**: Program kedua - counting FFB per class

In [ ]:
# Phase 5.1: Konfigurasi Tracker Parameter untuk Object Tracking
# Parameter dibawah adalah parameter yang digunakan dalam model.track(..., tracker='my_tracker.yaml')
# Parameter ini dapat di tune untuk menghasilkan object counting yang lebih baik
# Sumber: https://docs.ultralytics.com/modes/track/#tracker-arguments
%%writefile my_tracker.yaml

tracker_type: bytetrack        # memakai algoritma ByteTrack untuk tracking-by-detection
track_high_thresh: 0           # semua deteksi dianggap high-confidence (biasanya terlalu longgar)
track_low_thresh: 0            # semua deteksi, bahkan yang sangat rendah, ikut proses tracking
track_buffer: 300              # track boleh hilang sampai 300 frame sebelum dianggap mati
fuse_score: True               # score deteksi digabung dengan score tracking agar stabil
match_thresh: 0.9              # IoU harus ≥ 0.9 untuk bisa match dengan track lama (sangat ketat)
new_track_thresh: 0.85         # hanya deteksi dengan confidence ≥ 0.85 yang boleh bikin track baru

Overwriting my_tracker.yaml


In [ ]:
# ============================================================
# PART 5.2: Program untuk Counting Total FFB dalam Video
# ============================================================
# Program untuk counting total FFB yang ada dalam 1 video (VIDEO_PATH)
# Link video hasil program: https://drive.google.com/file/d/1wTA8am06jt54YaoO1t_E6V0sjhFk_nTl/view?usp=drive_link
import cv2
from ultralytics import YOLO

# Path model, video input, dan video output
MODEL_PATH = 'best.pt'
VIDEO_PATH = '/kaggle/working/drone_record_inference_test2.MP4'
OUTPUT_PATH = 'output_video_count.mp4'

# Jumlah frame minimum sebuah ID harus muncul agar dianggap stabil
MIN_FRAMES_TO_COUNT = 60

# Load model YOLO
model = YOLO(MODEL_PATH)

# Buka video input
cap = cv2.VideoCapture(VIDEO_PATH)

# Ambil properti video: width, height, fps
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Setup writer untuk menyimpan video output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (w, h))

# Dictionary untuk menyimpan total frame tiap ID muncul
track_history = {}

# Set berisi ID yang sudah dianggap stabil (muncul ≥ MIN_FRAMES_TO_COUNT)
stable_counted_ids = set()

print("Processing video with stability filter...")

# Loop membaca frame dari video
while cap.isOpened():
    success, frame = cap.read()
    if not success:         # berhenti kalau frame habis
        break
        
    # Jalankan tracking YOLO dengan tracker custom (my_tracker.yaml)
    results = model.track(frame, persist=True, verbose=False, tracker='my_tracker.yaml') 
    
    # Gambar bounding box + track ID pada frame
    annotated_frame = results[0].plot(
        line_width=2,
        font_size=2
    )
    
    # Jika ada ID yang terdeteksi pada frame ini
    if results[0].boxes.id is not None:
        track_ids = results[0].boxes.id.int().tolist()
        
        # Update count kemunculan tiap track ID
        for track_id in track_ids:
            if track_id not in track_history:
                track_history[track_id] = 1        # pertama kali muncul
            else:
                track_history[track_id] += 1       # tambah 1 frame
                
            # Jika ID sudah muncul cukup lama, masukkan ke stable list
            if track_history[track_id] >= MIN_FRAMES_TO_COUNT and track_id not in stable_counted_ids:
                stable_counted_ids.add(track_id)

    # Hitung total ID stabil
    total_stable_count = len(stable_counted_ids)
    
    # Teks yang ditampilkan pada video
    text = f'Total FFBs: {total_stable_count}'
    font_scale = 1.0
    thickness = 1
    
    # Hitung panjang teks untuk membuat background rectangle
    (text_w, text_h), _ = cv2.getTextSize(text, font, font_scale, thickness)
    
    # Background hitam untuk teks agar lebih terbaca
    cv2.rectangle(annotated_frame, (10, 10), (10 + text_w + 10, 10 + text_h + 10), (0, 0, 0), -1)
    
    # Tulis teks jumlah FFB pada frame
    cv2.putText(annotated_frame, text, (10 + 5, 10 + text_h + 5), font, font_scale, (255, 255, 255), thickness)
                
    # Simpan frame ke output video
    out.write(annotated_frame)

# Tutup resource video
cap.release()
out.release()

# Info tambahan setelah selesai
highest_id = max(stable_counted_ids)
print(f"✅ Processing complete!")
print(f"Highest stable ID: {highest_id}")
print(f"The 'highest' ID created during the entire video: {max(track_history.keys())}")
print(f"Final stable FFB count: {len(stable_counted_ids)}")
print(f"Video saved to: {OUTPUT_PATH}")

In [ ]:
# ============================================================
# PART 5.3: Program untuk Counting FFB dengan Per-Class Analysis
# ============================================================
# Program untuk counting total FFB beserta class-classnya
# Link video hasil program: https://drive.google.com/file/d/1X4ne58mU2VDyIZ9hF5hw9RghYg2UVBse/view?usp=drive_link
import cv2
from ultralytics import YOLO

# Path model, video input, dan video output
MODEL_PATH = 'best.pt'
VIDEO_PATH = '/kaggle/working/drone_record_inference_test2.MP4'
OUTPUT_PATH = 'output_video_count.mp4'

# Minimal jumlah frame sebelum sebuah track ID dianggap 'stabil'
MIN_FRAMES_TO_COUNT = 60

# Load model YOLO
model = YOLO(MODEL_PATH)

# Buka video input
cap = cv2.VideoCapture(VIDEO_PATH)

# Ambil properti video: width, height, fps
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Siapkan writer video output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (w, h))

# Daftar nama kelas dari model YOLO
class_names = model.names

# Dictionary riwayat track (jumlah frame + voting class)
track_history = {}

# Counter untuk setiap kelas (per-class counting)
class_counts = {name: 0 for name in class_names.values()}

# Set berisi ID yang sudah dianggap stabil (tidak dihitung 2x)
stable_counted_ids = set()

print("Processing video with per-class stability filter...")

# Loop membaca frame satu per satu
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break
        
    # Jalankan YOLO tracking per frame
    results = model.track(frame, persist=True, verbose=False, tracker='my_tracker.yaml') 
    
    # Gambar bounding box + ID + label pada frame
    annotated_frame = results[0].plot(
        line_width=2,  
        font_size=2    
    )
    
    # Cek apakah ada objek yang ditrack
    if results[0].boxes.id is not None:
        track_ids = results[0].boxes.id.int().tolist()     # daftar ID
        class_indices = results[0].boxes.cls.int().tolist()  # daftar class index
        
        # Loop untuk setiap objek yang muncul di frame
        for track_id, cls_index in zip(track_ids, class_indices):
            class_name = class_names[cls_index]  # nama kelas objek ini

            # Jika ID baru → mulai catat
            if track_id not in track_history:
                track_history[track_id] = {
                    'frame_count': 1,                     # muncul pertama kali
                    'class_votes': {class_name: 1}        # voting class pertama
                }
            else:
                # Tambah jumlah frame ia muncul
                track_history[track_id]['frame_count'] += 1
                
                # Voting class: pilih class yang paling sering muncul untuk ID ini
                votes = track_history[track_id]['class_votes']
                votes[class_name] = votes.get(class_name, 0) + 1

            # Jika sudah stabil & belum pernah dihitung → hitung
            if track_history[track_id]['frame_count'] >= MIN_FRAMES_TO_COUNT and track_id not in stable_counted_ids:
                stable_counted_ids.add(track_id)  # tandai sebagai sudah dihitung

                # Tentukan class final: class voting terbanyak
                votes = track_history[track_id]['class_votes']
                stable_class = max(votes, key=votes.get)
                
                # Tambahkan ke counter kelas tersebut
                class_counts[stable_class] += 1

    # Hitung total ID stabil
    total_stable_count = len(stable_counted_ids)
    
    # Siapkan teks laporan untuk ditampilkan pada video
    text_lines = [f'Total Stable FFBs: {total_stable_count}']
    for class_name, count in class_counts.items():
        if count > 0:
            text_lines.append(f'{class_name}: {count}')

    # Properti font
    font_scale = 1.0
    thickness = 2
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    # Menghitung ukuran dasar teks
    (text_w, text_h), _ = cv2.getTextSize('Test', font, font_scale, thickness)
    line_height = text_h + 10
    
    # Posisi kotak teks
    x_pos, y_pos = 10, 10
    
    # Cari width terbesar dari semua baris teks (supaya background pas)
    max_line_w = 0
    for line in text_lines:
        (line_w, _), _ = cv2.getTextSize(line, font, font_scale, thickness)
        max_line_w = max(max_line_w, line_w)

    # Hitung total ukuran background teks
    total_block_h = 10 + (line_height * len(text_lines)) - 5
    total_block_w = 10 + max_line_w + 10
    
    # Background hitam
    cv2.rectangle(annotated_frame, (x_pos, y_pos), (total_block_w, total_block_h), (0, 0, 0), -1)

    # Tulis teks baris per baris
    current_y = y_pos + text_h + 5
    for line in text_lines:
        cv2.putText(annotated_frame, line, (x_pos + 5, current_y), font, font_scale, (255, 255, 255), thickness)
        current_y += line_height
                
    # Simpan frame ke file output
    out.write(annotated_frame)

# Tutup file video
cap.release()
out.release()

print(f"✅ Processing complete!")

# Info tambahan hasil akhir
if stable_counted_ids:
    highest_id = max(stable_counted_ids)
    print(f"Highest stable ID: {highest_id}")
else:
    print("No stable IDs found.")

if track_history:
    print(f"The 'highest' ID created during the entire video: {max(track_history.keys())}")
else:
    print("No tracks created.")
    
print(f"Final total stable FFB count: {len(stable_counted_ids)}")
print("--- Final Counts Per Class ---")
print(class_counts)

print(f"Video saved to: {OUTPUT_PATH}")

Processing video with per-class stability filter...
✅ Processing complete!
Highest stable ID: 39
The 'highest' ID created during the entire video: 41
Final total stable FFB count: 38
--- Final Counts Per Class ---
{'Buah belum matang': 23, 'Buah busuk': 1, 'Buah matang': 8, 'Buah mau matang': 6}
Video saved to: output_video_count.mp4


In [ ]:
# Untuk download video di penyimpanan kaggle, klik link "vid.zip"
!zip vid.zip /kaggle/working/output_video_count.mp4
FileLink(r'vid.zip')

updating: kaggle/working/output_video_count.mp4 (deflated 1%)


/kaggle/working/vid.zip